In [103]:
import time
import json
import os
import socket

current_date = time.strftime('%d_%m_%Y')
filename = f'espsonda_{current_date}.json'

#filename = 'new_json_test.json'

# Check if the file already exists
if os.path.exists(filename) and os.path.getsize(filename) > 0 :
    # If the file exists, load its content
    with open(filename, 'r') as file:
        json_data = json.load(file)
else:
    # If the file doesn't exist, initialize the JSON structure
    json_data = {'epochtime': [], 'board': []}

#connect to espsonda
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = ('192.168.137.50', 2048)
print('connecting to %s port %s' % server_address)
sock.connect(server_address)

# Append new data multiple times
for x in range(6000):
    # Append the current epoch time
    epochtime = int(time.ctime())
    json_data['epochtime'].append(epochtime)

    message = '?'
    sock.sendall(message.encode(encoding='utf-8'))

    time.sleep(0.1)
    data = sock.recv(512).decode('utf-8')
    
    try:
        # Parse the received data as JSON
        received = json.loads(data)
        if "board" in received:
            json_data['board'].append(received["board"])
        else:
            print("Invalid data format: 'board' key not found.")
    except json.JSONDecodeError:
        print("Error decoding JSON from received data.")
    print("New data received.")

# Write the updated JSON data back to the file
with open(filename, 'w') as file:
    json.dump(json_data, file, indent=4)

print("Data updated succesfully.")

connecting to 192.168.1.177 port 2048
[{"id":0,"voltages":{"CH0":0.006905,"CH1":0.029493,"CH2":0.00181,"CH3":0.01798,"CH4":0.011047,"CH5":0.020605,"CH6":0.010534,"CH7":0.013822,"CH8":-0.002355,"CH9":-0.000294,"CH10":0.001571,"CH11":0.000063,"CH12":-0.007343,"CH13":0.007752,"CH14":-0.002569,"CH15":0.009623}}]
[{"id":0,"voltages":{"CH0":0.006905,"CH1":0.029493,"CH2":0.00181,"CH3":0.01798,"CH4":0.011047,"CH5":0.020605,"CH6":0.010534,"CH7":0.013822,"CH8":-0.002355,"CH9":-0.000294,"CH10":0.001571,"CH11":0.000063,"CH12":-0.007343,"CH13":0.007752,"CH14":-0.002569,"CH15":0.009623}}]
[{"id":0,"voltages":{"CH0":0.005035,"CH1":0.033763,"CH2":-0.000992,"CH3":0.016599,"CH4":0.009046,"CH5":0.02136,"CH6":0.008545,"CH7":0.015279,"CH8":0.000685,"CH9":0.00001,"CH10":-0.000932,"CH11":0.000127,"CH12":-0.004818,"CH13":0.00851,"CH14":-0.002524,"CH15":0.010588}}]
[{"id":0,"voltages":{"CH0":0.005035,"CH1":0.033763,"CH2":-0.000992,"CH3":0.016599,"CH4":0.009046,"CH5":0.02136,"CH6":0.008545,"CH7":0.015279,"CH8":

In [2]:
import pandas as pd
import matplotlib.pyplot  as plt
import json
#import numpy as np
from scipy.signal import savgol_filter
#import sg_filter

with open('espsonda_22_01_2025.json') as json_file:
    json_decoded = json.load(json_file)

# with open('espsonda_22_01_2025_10min.json') as json_file:
#     json_decoded = json.load(json_file)

# Assuming json_decoded["board"] contains all channel data
channels = [f"CH{i}" for i in range(16)]  # List of channels CH0 to CH15

# Extracting voltage values for each channel
channel_values = {channel: [entry[0]["voltages"].get(channel, None) for entry in json_decoded["board"]] for channel in channels}


# Creating the DataFrame
df = pd.DataFrame({
    "epochtime": json_decoded["epochtime"],
    **channel_values
})

# Convert epochtime to datetime
df['epochtime'] = pd.to_datetime(df['epochtime'], unit='s')

# Print the DataFrame
print(df)

# Plotting all channels
plt.figure(figsize=(20, 20))  # Adjust the figure size for better visibility

for i, channel in enumerate(channels):
    plt.subplot(4, 4, i + 1)  # 4x4 grid for 16 subplots
    plt.plot(df["epochtime"], df[channel], label=f"{channel} vs Time")
    plt.title(channel)
    plt.xlabel("Time")
    plt.ylabel("Voltage")
    plt.legend(loc=2)
    plt.tight_layout()

plt.show()

# Apply Savitzky-Golay filter to smooth the data
smoothed_data = {}
for channel in channels:
    signal = df[channel].dropna()  # Remove NaN values
    smoothed_signal = savgol_filter(signal, 401, 3)  # Window size 101, polynomial order 3
    smoothed_data[channel] = smoothed_signal

# Add smoothed data to DataFrame
for channel, smoothed_signal in smoothed_data.items():
    df[f"{channel}_smoothed"] = smoothed_signal

# Plot the original and smoothed data for each channel
plt.figure(figsize=(20, 20))  # Adjust the figure size for better visibility
for i, channel in enumerate(channels):
    plt.subplot(4, 4, i + 1)  # 4x4 grid for 16 subplots
    plt.plot(df["epochtime"], df[channel], linestyle='-', linewidth=2, alpha=0.5, label=f"{channel} (Original)")
    plt.plot(df["epochtime"], df[f"{channel}_smoothed"], color='r', label=f"{channel} (Smoothed)")
    plt.title(channel)
    plt.xlabel("Time")
    plt.ylabel("Voltage")
    plt.legend()
    plt.tight_layout()
plt.show()

# Sampling frequency (Hz) - adjust this based on your data
# fs = 10  # Example: 10 samples per second

# # Plotting the FFT of all channels
# plt.figure(figsize=(20, 20))  # Adjust the figure size for better visibility

# for i, channel in enumerate(channels):
#     # Remove NaN values from the channel for FFT
#     signal = df[channel].dropna()

#     # Perform FFT
#     n = len(signal)
#     freqs = np.fft.rfftfreq(n, d=1/fs)  # Frequency bins
#     fft_values = np.fft.rfft(signal)    # FFT values
#     magnitude = np.abs(fft_values)      # Magnitude of FFT
    
#     # Plot FFT
#     plt.subplot(4, 4, i + 1)  # 4x4 grid for 16 subplots
#     plt.plot(freqs, magnitude, label=f"FFT of {channel}")
#     plt.title(f"FFT of {channel}")
#     plt.xlabel("Frequency (Hz)")
#     plt.ylabel("Magnitude")
#     plt.legend()
#     plt.tight_layout()

# plt.show()



ModuleNotFoundError: No module named 'pandas'